In [1]:
from pycaret.classification import * 
import numpy as np
import pandas as pd
import seaborn as sns

In [7]:
df=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
test=test.drop(['ID'],axis=1)
test=test.reset_index(drop=True)
df=df.drop(['ID'],axis=1)
df.head()

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,188,1,0,1,0,0.0,0,1,0
1,209,1,0,1,0,0.0,0,2,1
2,257,1,0,1,0,0.0,0,2,1
3,257,1,1,1,0,0.0,0,2,1
4,342,1,0,1,0,0.0,0,2,1


In [3]:
data = df.sample(frac=0.8, random_state=1).reset_index(drop=True)
data_unseen = df.drop(data.index).reset_index(drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions ' + str(data_unseen.shape))

Data for Modeling: (71086, 9)
Unseen Data For Predictions (17772, 9)


In [5]:
exp = setup(data = data, target = 'Crop_Damage', session_id=1,
                  normalize = True,
                categorical_features = ['Crop_Type','Soil_Type','Pesticide_Use_Category','Season'],
                numeric_features=['Estimated_Insects_Count','Number_Doses_Week','Number_Weeks_Quit','Number_Weeks_Used'],
                categorical_imputation='mode',
                numeric_imputation='mean',
                  transformation = True, 
                    pca=True,
            remove_outliers=True,
            feature_selection=True
            
                )

Setup Succesfully Completed!


,Description,Value
0,session_id,1
1,Target Type,Multiclass
2,Label Encoded,None
3,Original Data,"(71086, 9)"
4,Missing Values,True
5,Numeric Features,4
6,Categorical Features,4
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [6]:
compare_models()#this is with normalize and transform

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Light Gradient Boosting Machine,0.8506,0.0000,0.3725,0.7966,0.8051,0.1661,0.2235,0.9140
1,CatBoost Classifier,0.8497,0.0000,0.3794,0.8001,0.8079,0.1839,0.2328,47.1374
2,Extreme Gradient Boosting,0.8496,0.0000,0.3633,0.7921,0.7992,0.1323,0.1970,1.9065
3,Gradient Boosting Classifier,0.8491,0.0000,0.3655,0.7920,0.8006,0.1416,0.2015,33.1243
4,Ada Boost Classifier,0.8468,0.0000,0.3626,0.7858,0.7975,0.1259,0.1801,7.3175
5,SVM - Linear Kernel,0.8453,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000,0.2050
6,Ridge Classifier,0.8452,0.0000,0.3341,0.7545,0.7752,0.0037,0.0212,0.0528
7,Logistic Regression,0.8444,0.0000,0.3579,0.7789,0.7937,0.1076,0.1566,0.1473
8,Naive Bayes,0.8443,0.0000,0.3680,0.7832,0.7999,0.1457,0.1883,0.0420
9,Linear Discriminant Analysis,0.8431,0.0000,0.3672,0.7814,0.7989,0.1414,0.1820,0.1748


OneVsRestClassifier(estimator=LGBMClassifier(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             learning_rate=0.1, max_depth=-1,
                                             min_child_samples=20,
                                             min_child_weight=0.001,
                                             min_split_gain=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             num_leaves=31, objective=None,
                                             random_state=1, reg_alpha=0.0,
                                             reg_lambda=0.0, silent=True,
                                             subsample=1.0,
                                             subsample_for_bin=200000,
                      

In [8]:
lr=create_model('lr')
tuned_lr=tune_model(lr,optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8450,0.0000,0.3588,0.7806,0.7943,0.1106,0.1617
1,0.8435,0.0000,0.3549,0.7766,0.7911,0.0924,0.1396
2,0.8454,0.0000,0.3580,0.7800,0.7943,0.1104,0.1623
3,0.8439,0.0000,0.3569,0.7769,0.7930,0.1050,0.1526
4,0.8432,0.0000,0.3580,0.7756,0.7938,0.1122,0.1570
5,0.8454,0.0000,0.3598,0.7822,0.7952,0.1135,0.1649
6,0.8439,0.0000,0.3555,0.7767,0.7920,0.0980,0.1462
7,0.8409,0.0000,0.3539,0.7702,0.7899,0.0912,0.1310
8,0.8445,0.0000,0.3585,0.7804,0.7939,0.1071,0.1565
9,0.8483,0.0000,0.3651,0.7901,0.7995,0.1360,0.1944


In [9]:
lgbm=create_model('lightgbm')
tuned_lgbm=tune_model(lgbm,optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8484,0.0000,0.3774,0.7929,0.8070,0.1820,0.2278
1,0.8466,0.0000,0.3749,0.7907,0.8044,0.1657,0.2104
2,0.8468,0.0000,0.3736,0.7888,0.8046,0.1711,0.2151
3,0.8494,0.0000,0.3831,0.8032,0.8086,0.1891,0.2355
4,0.8475,0.0000,0.3697,0.7876,0.8027,0.1585,0.2073
5,0.8528,0.0000,0.3860,0.8024,0.8133,0.2095,0.2594
6,0.8538,0.0000,0.3804,0.8023,0.8120,0.2034,0.2595
7,0.8483,0.0000,0.3692,0.7912,0.8024,0.1529,0.2061
8,0.8490,0.0000,0.3772,0.7944,0.8068,0.1763,0.2249
9,0.8473,0.0000,0.3777,0.7968,0.8044,0.1673,0.2143


# Trying Ensembling

In [10]:
bagged_lightgbm = ensemble_model(tuned_lgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8509,0.0000,0.3771,0.7960,0.8081,0.1850,0.2377
1,0.8492,0.0000,0.3772,0.7951,0.8068,0.1754,0.2249
2,0.8481,0.0000,0.3732,0.7899,0.8049,0.1703,0.2180
3,0.8511,0.0000,0.3784,0.7961,0.8091,0.1905,0.2414
4,0.8487,0.0000,0.3725,0.7911,0.8047,0.1669,0.2175
5,0.8483,0.0000,0.3746,0.7918,0.8057,0.1737,0.2212
6,0.8564,0.0000,0.3833,0.8073,0.8141,0.2096,0.2726
7,0.8487,0.0000,0.3711,0.7919,0.8038,0.1608,0.2133
8,0.8494,0.0000,0.3746,0.7937,0.8058,0.1710,0.2225
9,0.8483,0.0000,0.3751,0.7919,0.8057,0.1745,0.2223


In [11]:
svm=create_model('svm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8452,0.0000,0.3333,0.7143,0.7743,0.0000,0.0000
1,0.8454,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000
2,0.8454,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000
3,0.8454,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000
4,0.8454,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000
5,0.8454,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000
6,0.8454,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000
7,0.8454,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000
8,0.8454,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000
9,0.8451,0.0000,0.3333,0.7143,0.7742,0.0000,0.0000


In [12]:
boosted_svm = ensemble_model(svm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8452,0.0000,0.3333,0.7143,0.7743,0.0000,0.0000
1,0.8454,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000
2,0.8454,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000
3,0.8454,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000
4,0.8454,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000
5,0.8454,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000
6,0.8454,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000
7,0.8454,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000
8,0.8454,0.0000,0.3333,0.7146,0.7745,0.0000,0.0000
9,0.8451,0.0000,0.3333,0.7143,0.7742,0.0000,0.0000


In [13]:
unseen_predictions = predict_model(bagged_lightgbm, data=test)
unseen_predictions.head()

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Label,Score
0,188,1,1,1,0,NaN,0,2,0,0.9932
1,410,1,1,1,0,0.0,0,2,0,0.9840
2,626,1,0,1,0,0.0,0,2,0,0.9915
3,731,1,0,1,0,0.0,0,2,0,0.9874
4,789,0,0,1,0,0.0,0,1,0,0.9744


In [15]:
unseen_predictions.to_csv('submissionfile.csv')